In [6]:
import re
import time
import random
import requests
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities

# 트위터 고급 검색 기능 활용
- 일별로 판다스 시리즈 url 만들어서 크롤 해보기
## 트럼프
### 트럼프 본인 작성 트윗
(from:realDonaldTrump) lang:en until:2020-10-31 since:2020-09-01
- https://twitter.com/search?q=(from%3ArealDonaldTrump)%20lang%3Aen%20until%3A2020-10-31%20since%3A2020-09-01&src=typed_query
### 관중 -> 트럼프       
(to:realDonaldTrump) (@realDonaldTrump) lang:en until:2020-10-31 since:2020-09-01
- https://twitter.com/search?q=(to%3ArealDonaldTrump)%20(%40realDonaldTrump)%20lang%3Aen%20until%3A2020-10-31%20since%3A2020-09-01&src=typed_query

## 바이든
### 바이든 본인 작성 트윗
(from:JoeBiden) lang:en until:2020-10-31 since:2020-09-01
- https://twitter.com/search?q=(from%3AJoeBiden)%20lang%3Aen%20until%3A2020-10-31%20since%3A2020-09-01&src=typed_query
### 관중 -> 바이든       
(to:JoeBiden) (@JoeBiden) lang:en until:2020-10-31 since:2020-09-01
- https://twitter.com/search?q=(to%3AJoeBiden)%20(%40JoeBiden)%20lang%3Aen%20until%3A2020-10-31%20since%3A2020-09-01&src=typed_query

# URL 추출하기

In [7]:
# import datetime as dt
# search_by_day_list=[]
# URLs={'Trump':[],'Biden':[],'ppl2Trump':[],'ppl2Biden':[]}
# startdate=dt.date(year=2020,month=9,day=1)
# untildate=dt.date(year=2020,month=9,day=2)
# # enddate=dt.date(year=2020,month=10,day=31)
# # 10월 31일까지 읽으려면 11월2일까지의 인덱스길이가 있어야한다.
# while untildate != dt.date(year=2020,month=11,day=2):
#     url_Trump='https://twitter.com/search?q=(from%3ArealDonaldTrump)%20lang%3Aen%20until%3A'+str(untildate)+'%20since%3A'+str(startdate)+'&src=typed_query'
#     url_Biden='https://twitter.com/search?q=(from%3AJoeBiden)%20lang%3Aen%20until%3A'+str(untildate)+'%20since%3A'+str(startdate)+'&src=typed_query'
#     url_ppl2Trump='https://twitter.com/search?q=(to%3ArealDonaldTrump)%20(%40realDonaldTrump)%20lang%3Aen%20until%3A'+str(untildate)+'%20since%3A'+str(startdate)+'&src=typed_query'
#     url_ppl2Biden='https://twitter.com/search?q=(to%3AJoeBiden)%20(%40JoeBiden)%20lang%3Aen%20until%3A'+str(untildate)+'%20since%3A'+str(startdate)+'&src=typed_query'
#     startdate=untildate
#     untildate+=dt.timedelta(days=1)
#     URLs['Trump'].append(url_Trump)
#     URLs['Biden'].append(url_Biden)
#     URLs['ppl2Trump'].append(url_ppl2Trump)
#     URLs['ppl2Biden'].append(url_ppl2Biden)

In [27]:
import datetime as dt
search_by_day_list=[]
URLs={'삼성전자':[]}
startdate=dt.date(year=2019,month=1,day=1)
untildate=dt.date(year=2019,month=1,day=2)
# enddate=dt.date(year=2020,month=10,day=31)
# 10월 31일까지 읽으려면 11월2일까지의 인덱스길이가 있어야한다.
while untildate != dt.date(year=2021,month=12,day=31):
    url_samsung='https://twitter.com/search?q=%3A삼성전자%20lang%3Ako%20until%3A'+str(untildate)+'%20since%3A'+str(startdate)+'&src=typed_query'
    startdate=untildate
    untildate+=dt.timedelta(days=1)
    URLs['삼성전자'].append(url_samsung)

In [28]:
# 삼성전자 현대제철 삼성바이오로직스 우리은행 대한항공 팬오션 

In [29]:
url_df=pd.DataFrame(URLs)
url_df

,삼성전자
0,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
1,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
2,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
3,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
4,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
...,...
1089,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
1090,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
1091,https://twitter.com/search?q=%3A삼성전자%20lang%3A...
1092,https://twitter.com/search?q=%3A삼성전자%20lang%3A...


# 각각의 column별로 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from progressbar import ProgressBar
from time import sleep
import random

browser_path = "C:/Users/ym/chromedriver/chromedriver.exe" # setting the chromedriver path and initializing driver
driver = webdriver.Chrome(executable_path=browser_path)   # create master df to append to
master_temp_df = pd.DataFrame()
master_df = pd.DataFrame()

def sleep_for(opt1, opt2):                                # random delay-time generator
    time_for = random.uniform(opt1, opt2)
    time_for_int = int(round(time_for))
    sleep(abs(time_for_int - time_for))
    for i in range(time_for_int, 0, -1):
        sleep(1)
        
# 주제별로 url시리즈 넣어서 일별로 크롤링
#10월
for urls in url_df['삼성전자']:
    pbar = ProgressBar()
    urls=[urls]
    post_element_xpath = '//div/div/article/div/div' # the element we are obtaining from the webpage
    for url in pbar(urls):                          # loop through your list of urls
        driver.get(url)
        sleep_for(3, 7)                             # sleep a while

        lastHeight = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.find_element_by_xpath('//body').send_keys(Keys.END)
            sleep_for(2, 6)
            newHeight = driver.execute_script("return document.body.scrollHeight")
            if newHeight != lastHeight:
                pass
            else:
                post_list = driver.find_elements_by_xpath(post_element_xpath)      # get a list of tweets
                post_text = [x.text for x in post_list]                           # get the text only from each element
                temp_df = pd.DataFrame(post_text, columns={'all_text'})           # create temporary dataset of each tweet
                master_temp_df = master_temp_df.append(temp_df)                   # append the temporary dataset to the dataset we will save
                break
            
            lastHeight = newHeight      # update Height
            master_df = master_df.append(master_temp_df)

            
driver.quit()     
master_df.to_excel("삼성전자.xlsx")

C:\Users\ym\AppData\Local\Temp/ipykernel_13932/1696386725.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=browser_path)   # create master df to append to
C:\Users\ym\AppData\Local\Temp/ipykernel_13932/1696386725.py:33: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//body').send_keys(Keys.END)
C:\Users\ym\AppData\Local\Temp/ipykernel_13932/1696386725.py:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  post_list = driver.find_elements_by_xpath(post_element_xpath)      # get a list of tweets
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##############################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |########################################################################|
100% |##################################